In [1]:
import pandas as pd  
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.linear_model import LogisticRegression 
from sklearn.metrics import classification_report
import re 
import joblib
import string

In [2]:
fake = pd.read_csv('Fake.csv')
true = pd.read_csv('True.csv')

In [3]:
fake.head()

,title,text,subject,date
0,Donald Trump Sends Out Embarrassing New Year’...,Donald Trump just couldn t wish all Americans ...,News,"December 31, 2017"
1,Drunk Bragging Trump Staffer Started Russian ...,House Intelligence Committee Chairman Devin Nu...,News,"December 31, 2017"
2,Sheriff David Clarke Becomes An Internet Joke...,"On Friday, it was revealed that former Milwauk...",News,"December 30, 2017"
3,Trump Is So Obsessed He Even Has Obama’s Name...,"On Christmas day, Donald Trump announced that ...",News,"December 29, 2017"
4,Pope Francis Just Called Out Donald Trump Dur...,Pope Francis used his annual Christmas Day mes...,News,"December 25, 2017"


In [4]:
true.head()

,title,text,subject,date
0,"As U.S. budget fight looms, Republicans flip t...",WASHINGTON (Reuters) - The head of a conservat...,politicsNews,"December 31, 2017"
1,U.S. military to accept transgender recruits o...,WASHINGTON (Reuters) - Transgender people will...,politicsNews,"December 29, 2017"
2,Senior U.S. Republican senator: 'Let Mr. Muell...,WASHINGTON (Reuters) - The special counsel inv...,politicsNews,"December 31, 2017"
3,FBI Russia probe helped by Australian diplomat...,WASHINGTON (Reuters) - Trump campaign adviser ...,politicsNews,"December 30, 2017"
4,Trump wants Postal Service to charge 'much mor...,SEATTLE/WASHINGTON (Reuters) - President Donal...,politicsNews,"December 29, 2017"


In [5]:

fake['class']=0
true['class']=1

In [6]:
data = pd.concat([fake,true], axis = 0)

In [7]:
data.sample(10)

,title,text,subject,date,class
6321,Racist Cowards Nearly Beat Black Ex-Marine To...,Clayton Denny is a former U.S. Marine who move...,News,"May 17, 2016",0
1888,"U.S. states, immigrant groups to fight Trump's...",WASHINGTON (Reuters) - Several U.S. states and...,politicsNews,"September 5, 2017",1
20228,DEMS BOOED GOD At The Last Convention…This Tim...,No respect for God No respect for our Constitu...,left-news,"Jul 25, 2016",0
3677,How Obama And Trump Responded To Fidel Castro...,President Obama couldn t be more different fro...,News,"November 26, 2016",0
1282,London Mayor Just Told Trump To F*ck Off In T...,"After the Paris Accord, the United States unde...",News,"June 4, 2017",0
3028,Trump Just Got Ripped To F*cking Shreds After...,Between Trump s daily tantrums before I ve eve...,News,"January 13, 2017",0
3296,Montana congressman-elect to be sentenced for ...,(Reuters) - Montana Republican congressman-ele...,politicsNews,"June 8, 2017",1
6302,Mitt Romney’s Effort To Stop Trump Ends Just ...,"Man, it has got to suck to be Mitt Romney. We ...",News,"May 18, 2016",0
3759,"Whoopi Goldberg Gets PISSED, Gives Trump The ...",The appointment of several white supremacists ...,News,"November 18, 2016",0
3049,Sam Bee On #GoldenShowerGate: ‘It’s Number On...,Trump has had a rough week! We d feel sorry fo...,News,"January 12, 2017",0


In [8]:
data = data.drop(["title", "subject", "date"], axis = 1)

In [9]:
data.reset_index(inplace=True)


In [10]:
data.drop(['index'], axis = 1, inplace=True)

In [11]:
data.sample(5)

,text,class
19493,,0
38133,ROME (Reuters) - Italy s ruling Democratic Par...,1
19387,President Barack Obama designated two national...,0
33438,(Reuters) - The Obama administration will anno...,1
23617,WASHINGTON (Reuters) - The Trump administratio...,1


In [12]:
def clean_text(text):
    text = text.lower()
    text = re.sub('\[.*?\]',"",text)
    text = re.sub("\\W"," ",text)
    text = re.sub("https?:://\S+|www\.\S+","",text)
    text = re.sub("<.*?>+","",text)
    text = re.sub("[%s]" % re.escape(string.punctuation),"",text)
    text = re.sub("\n","",text)
    text = re.sub("\w*\d\w*","",text)
    return text 

In [13]:
data["text"] = data["text"].apply(clean_text)

In [14]:
x=data["text"]
y=data["class"]

xtrain, xtest, ytrain, ytest = train_test_split(x,y,test_size=0.25, random_state=42)


In [15]:
vectorizer = TfidfVectorizer()
xv_train = vectorizer.fit_transform(xtrain)
xv_test = vectorizer.transform(xtest) 

In [16]:
lr = LogisticRegression()
lr.fit(xv_train,ytrain)

LogisticRegression()

In [17]:
prediction = lr.predict(xv_test)
print("Accuracy:", lr.score(xv_test, ytest))

Accuracy: 0.9858351893095768


In [18]:
print(classification_report(ytest,prediction))

              precision    recall  f1-score   support

           0       0.99      0.99      0.99      5895
           1       0.98      0.99      0.99      5330

    accuracy                           0.99     11225
   macro avg       0.99      0.99      0.99     11225
weighted avg       0.99      0.99      0.99     11225



In [19]:
joblib.dump(vectorizer,"vectorizer.jb")
joblib.dump(lr,"lr_model.jb")

['lr_model.jb']